In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from string import digits
from collections import Counter

from tqdm import tqdm

from sklearn.model_selection import train_test_split

사용할 데이터 업로드 : total_data

In [49]:
total_data=pd.read_excel('D:\\KMI-Project_tmp\\Data\\Sentimental\\Naver\\General\\TotalDoc_General.xlsx',usecols=['Content','Label']) ##전체 데이터 

Total Data 전처리: 
- 부정 값 0으로 변경
- content 내용 문장 단위로 split

In [50]:
total_data['Content'].nunique(), total_data['Label'].nunique() #TotalDataSet에 중복 확인 
total_data.dropna(subset=['Content'],inplace=True)
total_data['Content']=total_data['Content'].str.strip()
total_data.drop_duplicates(subset=['Content'], inplace=True) # reviews 열에서 중복인 내용이 있다면 중복 제거
print('총 샘플의 수 :',len(total_data))
print('null 값 유무: ',total_data.isnull().values.any())

총 샘플의 수 : 62258
null 값 유무:  False


In [51]:
total_len=0
count_len=0
max_len=0

for i in total_data['Content']:
    tmp_split=i.split('다 ')#tmp_split 리스트 생성

    for j in tmp_split:
        count_len+=1
        total_len+=len(j)

        if max_len<len(j) and len(j)<1000:
            max_len=len(j)
            max_content=j

In [52]:
print('기사 한 문장의 평균 길이:',total_len/count_len)
print('기사 한 문장의 최대 길이:',max_len)
print(max_content)

기사 한 문장의 평균 길이: 48.21530607999729
기사 한 문장의 최대 길이: 988
한국을 오가면서 학회 토론도 하고 발표도 했지만 제가 이 있진 않았기 관찰자 입장이었어요 제가 연구를 계기가 있어요 프랑스 보르도 대학에 있을 학생들이 한국 문화 동아리를 하고 있었는데 저는 이게 유학생들이 주축이 된 동아리인 줄 알았어요 옛날에 우리 세대 미국 유학생들이 한국이 그리워서 같이 모여서 한국 드라마 보고 그랬던 것처럼 말이죠 그런데 아니더라고요 한국 학생이 한 명도 없고 모두 프랑스 학생들이었어요 그렇다고 보르도 대학이 파리나 마르세유 대학처럼 다문화적인 곳도 아니거든요 그래서 흥미가 생겼죠 이 학생들이 모여서 뭘 하나 했더니 한국 드라마를 보고 있더라고요 그래서 저도 이 학생들을 인터뷰하고 드라마를 보기 한국을 떠나서 못 봤던 드라마들 그 쌓인 것을 보면서 너무 놀랐어요 한국 드라마의 질적인 성장이 보였거든요 그리고 정말 글로벌한 문화의 유통 매개자가 필요 없는 자발적인 수용 언어의 장벽을 아무 문제 없이 뛰어넘는 자발적 번역 자막 subtitle 이라고 하잖아요 그런 조직화된 문화 팬덤이 있다는 걸 확인한 거잖아요 그래서 그 한국 드라마 보는 프랑스 팬덤 동아리에 들어가서 이런 환경이 어떻게 이뤄졌는지 왜 한국 드라마를 보는지 거의 정도 관찰하고 대화하고 연구했어요 여기서 한류의 흐름 이야기로 넘어갈 수 있을 거 같은데요 저는 그 한국 드라마는 넘어 전세계에서 볼 수 있는 내용이었고 질적으로도 잘 만들고 있었기 앞으로 더 커질 것이라고 사실 한국이 경쟁력 있게 만드는 게 로맨틱 코미디예요 그러니까 남자 여자 알콩달콩 연애하는 건데 그건 정말 늘 비슷한 이야기지만 매번 재밌잖아요 그건 제작 능력이 좋고 연기도 잘한다는 얘기죠 요즘 한국 드라마 중에 전세계에서 인기를 끈 건 킹덤 이라든지 오징어게임 같은 거였는데 그건 글로벌 OTT인 넷플릭스가 개입을 한 거고 우리가 주력했던 장르들은 아니에요 한류는 그 개념 자체가 한국 밖에서 한국 대중문화의 인기를

In [53]:
mean_len=50 #한 기사의 문장 길이를 mean_len으로 설정하고 split된 sentence의 길이 표준화 

In [54]:
def make_list(content_list):
    tmp_list=[]
    sentence_list=[]

    for sentence in content_list:
        if len(sentence)>mean_len: sentence_list.append(sentence)
        else:
            if not tmp_list: tmp_list.append(sentence)
            else:
                tmp_sentence=' '.join(tmp_list)
                
                if len(tmp_sentence)>mean_len: 
                    sentence_list.append(tmp_sentence)
                    
                    tmp_list=[]
                    tmp_list.append(sentence)
                else:
                    tmp_list.append(sentence)
                    
    return sentence_list

In [55]:
def split(contents):
    sentence_list=[]
    
    split_contents=contents.split('다 ')
    split_contents=split_contents[:len(split_contents)-1]
    
    sentence_list=make_list(split_contents)
    
    return sentence_list

In [56]:
def match(sentence_list,label):
    tmp_sentence=[]
    tmp_label=[]
    
    for i in sentence_list:
        tmp_sentence.append(i)
        tmp_label.append(label)
    
    df_tmp=pd.DataFrame(
        {'Sentence':tmp_sentence,
         'Label':tmp_label}
    )
    
    return df_tmp

In [57]:
df_total=pd.DataFrame(index=['Sentence','Label'])

for i in tqdm(total_data['Content']):
    label=total_data.loc[total_data['Content']==i,['Label']].values[0][0]
    sentence_list=split(i); df_tmp=match(sentence_list,label)
    df_total=df_total.append(df_tmp,ignore_index=True)

100%|█████████████████████████████████████████████████| 62258/62258 [48:10<00:00, 21.54it/s]


In [58]:
def count_len(x):
    try:
        return len(x)
    except: return 0

In [59]:
text_len = df_total['Sentence'].map(lambda x: count_len(x))
np.mean(text_len)

76.22138370787859

In [60]:
df_total['Sentence'].nunique(), df_total['Label'].nunique() #TotalDataSet에 중복 확인 
df_total.dropna(subset=['Sentence'],inplace=True)
df_total['Sentence']=df_total['Sentence'].str.strip()
df_total.drop_duplicates(subset=['Sentence'], inplace=True) # reviews 열에서 중복인 내용이 있다면 중복 제거

print('총 샘플의 수 :',len(df_total))
print('null 값 유무: ',df_total.isnull().values.any())

총 샘플의 수 : 739634
null 값 유무:  False


In [42]:
df_total['Label']=df_total['Label'].astype(int)
df_total['Label']=df_total['Label'].apply(lambda x: 0 if x==-1 else x) #부정 레이블 0으로 설정

데이터 split: train_data, test_data( 50:50 비율)

In [61]:
train_data,test_data=train_test_split(df_total,test_size=0.25,random_state=42) 
print('훈련 용 리뷰의 개수: ',len(train_data)); print(train_data.groupby('Label').size().reset_index(name = 'count'))
print('테스트용 리뷰의 개수: ',len(test_data)); print(test_data.groupby('Label').size().reset_index(name = 'count'))   

훈련 용 리뷰의 개수:  554725
   Label   count
0   -1.0  418219
1    1.0  136506
테스트용 리뷰의 개수:  184909
   Label   count
0   -1.0  139110
1    1.0   45799


In [62]:
train_data.to_csv('D:\\KMI-Project_tmp\\Data\\BiLSTM_Model\\General\\Model_train_data.csv')
test_data.to_csv('D:\\KMI-Project_tmp\\Data\\BiLSTM_Model\\General\\Model_test_data.csv')